In [63]:
## Importing libraries
import pandas as pd
import pyreadstat
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Get a first look at the data

In [64]:
### Import Labor force survey 
## 2556 Q1
LFS_56_Q1, meta_56_Q1 = pyreadstat.read_sav("Data\LFS\LFS 2556 (2013)\LFS Q1 2556\Microdata LFS Q1 2556\Microdata LFS Q1 2556.sav")

In [65]:
LFS_56_Q1

,reg,cwt,area,psu_no,ea_set,samset,mounth,yr,hh_no,member,...,re_no,wage_ty,amount,approx,bonus,ot,oth_mon,re_wk,re_ed,wgt
0,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,11.0,1065.8194
1,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,926.1319
2,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,1.0,3.0,...,NaN,4.0,NaN,6000.0,0.0,0.0,0.0,1.0,3.0,1327.5656
3,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,713.0996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226611,5.0,96.0,2.0,54.0,3.0,B,3.0,56.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,503.8714
226612,5.0,96.0,2.0,54.0,3.0,B,3.0,56.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,460.0770
226613,5.0,96.0,2.0,54.0,3.0,B,3.0,56.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,393.3147
226614,5.0,96.0,2.0,54.0,3.0,B,3.0,56.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,376.1351


In [66]:
### What does each column mean
meta_56_Q1.column_names_to_labels

{'reg': None,
 'cwt': None,
 'area': None,
 'psu_no': None,
 'ea_set': None,
 'samset': None,
 'mounth': None,
 'yr': None,
 'hh_no': None,
 'member': None,
 'listing': None,
 'enum': None,
 'no': None,
 'sex': None,
 'age': None,
 'marital': None,
 'grade_a': None,
 'grade_b': None,
 'subject': None,
 'line': None,
 'wk_7day': None,
 'receive': None,
 'occup1': None,
 'occup2': None,
 'occup3': None,
 'occup': None,
 'ind1': None,
 'ind2': None,
 'ind3': None,
 'ind4': None,
 'indus': None,
 'status': None,
 'size': None,
 'main_hr': None,
 'other_hr': None,
 'total_hr': None,
 'more_wk': None,
 'more_hr': None,
 'finding': None,
 're_no': None,
 'wage_ty': None,
 'amount': None,
 'approx': None,
 'bonus': None,
 'ot': None,
 'oth_mon': None,
 're_wk': None,
 're_ed': None,
 'wgt': None}

In [67]:
## Label of each column
meta_56_Q1.variable_to_label

{}

In [68]:
### Value labels (What does each value in each column mean)

meta_56_Q1.value_labels

{}

### Keeping Only Labor Force

Value labels of 're_wk': 'WORK STATUS RECODE'

For 2528 - 2543:
- 01-08: Employed
- 09-12, 14-17: Unemployed
- **19-20, 22-26: Not in labor force** &rarr; 01-18, 21: Labor force
- 13, 18, 21: Waiting for season.

**Note**: must also exclude people age under 15 years old manually.

For 2544 - 2550:
- 01-03: Employed
- 04-05: Unemployed
- **06-07, 09-12: Not in labor force** &rarr; 01-05, 08: Labor force
- 08: Waiting for season.
- 13: Persons under 15 years old

For 2551 - recent:
- 01-03: Employed
- 04-05: Unemployed
- **06-07, 09-13: Not in labor force** &rarr; 01-05, 08: Labor force
- 08: Waiting for season.
- 14: Persons under 15 years old



In [69]:
### Crete functions to exclude people who are not in labor force.
## Excluding people who are not in labor force (2544 - recent)
def exclude_not_LFfrom44(LFS_df): # drop re_wk == 06-07 or >= 09.
    x = LFS_df.drop(LFS_df[(LFS_df["re_wk"] == 6) | (LFS_df["re_wk"] == 7) | (LFS_df["re_wk"] >= 9)].index)
    return x

## Excluding people who are not in labor force (before 2544)
def exclude_not_LFbefore44(LFS_df): # drop re_wk == 19-20 or 22-26
    x = LFS_df.drop(LFS_df[(LFS_df["re_wk"] == 19) | (LFS_df["re_wk"] == 20) | (LFS_df["re_wk"] == 22) | (LFS_df["re_wk"] == 23) | (LFS_df["re_wk"] == 24) | (LFS_df["re_wk"] == 25) | (LFS_df["re_wk"] == 26)].index)
    # drop those under 15 years old
    x = x.drop(x[x["age"] < 15].index)
    return x

In [70]:
## Apply the function to the data (use pipe since the input is a dataframe)
LFS_56_Q1 = LFS_56_Q1.pipe(exclude_not_LFfrom44)

In [71]:
## Check the total labor force (sum wgt)
LFS_56_Q1["wgt"].sum()

# correct!

38501921.0863

## Recoding Variables

### Coding Sector.

From *indus* variable

From 2528 - 2543, the standard is ISIC Rev (4 digits)1:
- Agriculture: 01-04
- Industry: 11-52
- Services: 61-90

From 2544 - 2553, the standard is ISIC Rev 3. (4 digits):
- Agriculture: 01-05
- Industry: 10-45
- Services: 50-99

For 2554, the standard is TSIC 2009 but 4 digits:
- Agriculture: 01-03
- Industry: 05-43
- Services: 45-99

From 2555 - recent years, the standard is TSIC 2009 (5 digits):
- Agriculture: 01-03
- Industry: 05-43
- Services: 45-99

In [72]:
### Define function to code SECTOR (from the first two digits of INDUST)

## function for 2528-2543.
def sector_create_28to43(row): 
    # First, convert INDUST from float to string.
    x = str(row["indus"])
    # remove the decimal part.
    x = x.split(".")[0]

    # If x is missing, return missing.
    if x == "nan":
        return np.nan
    
    ## In case x is less than 4 digits, add 0 in front. And use the first two digits.
    elif len(x) < 4:
        x = "0" + x
        x = x[:2]
        ## Convert the string back to integer.
        x = int(x)
    
    ## If it is already 4 digits, take it as it is.
    elif len(x) == 4:
        x = x[:2]
        ## Convert the string back to integer.
        x = int(x)

    ## Create the SECTOR code.
    # Agriculture: 01-04
    if (x >= 1) & (x <= 4):
        return "Agriculture"
    # Industry: 11-52
    elif (x >= 11) & (x <= 52):
        return "Industry"
    # Services: 61-90
    elif (x >= 61) & (x <= 90):
        return "Services"
    

## function for 2544-2553.
def sector_create_44to53(row):
    # First, convert INDUST from float to string.
    x = str(row["indus"])
    # remove the decimal part.
    x = x.split(".")[0]

    # If x is missing, return missing.
    if x == "nan":
        return np.nan
    
    ## In case x is less than 4 digits, add 0 in front. And use the first two digits.
    elif len(x) < 4:
        x = "0" + x
        x = x[:2]
        ## Convert the string back to integer.
        x = int(x)
    
    ## If it is already 4 digits, take it as it is.
    elif len(x) == 4:
        x = x[:2]
        ## Convert the string back to integer.
        x = int(x)

    ## Create the SECTOR code.
    # Agriculture: 01-05
    if (x >= 1) & (x <= 5):
        return "Agriculture"
    # Industry: 10-45
    elif (x >= 10) & (x <= 45):
        return "Industry"
    # Services: 50-99
    elif (x >= 50) & (x <= 99):
        return "Services"


## function for 2554.
def sector_create_54(row):
    # First, convert INDUST from float to string.
    x = str(row["indus"])
    # remove the decimal part.
    x = x.split(".")[0]

    # If x is missing, return missing.
    if x == "nan":
        return np.nan

    ## In case x is less than 4 digits, add 0 in front. And use the first two digits.
    elif len(x) < 4:
        x = "0" + x
        x = x[:2]
        ## Convert the string back to integer.
        x = int(x)
    
    ## If it is already 4 digits, take it as it is.
    elif len(x) == 4:
        x = x[:2]
        ## Convert the string back to integer.
        x = int(x)

    ## Create the SECTOR code.
    # Agriculture: 01-03
    if (x >= 1) & (x <= 3):
        return "Agriculture"
    # Industry: 05-43
    elif (x >= 5) & (x <= 43):
        return "Industry"
    # Services: 45-99
    elif (x >= 45) & (x <= 99):
        return "Services"
    

## function for 2555-recent.
def sector_create_from55(row):
    # First, convert INDUST from float to string.
    x = str(row["indus"])
    # remove the decimal part.
    x = x.split(".")[0]

    # If x is missing, return missing.
    if x == "nan":
        return np.nan
    
    ## In case x is less than 5 digits, add 0 in front. And use the first two digits.
    elif len(x) < 5:
        x = "0" + x
        x = x[:2]
        ## Convert the string back to integer.
        x = int(x)
    
    ## If it is already 5 digits, take it as it is.
    elif len(x) == 5:
        x = x[:2]
        ## Convert the string back to integer.
        x = int(x)

    ## Create the SECTOR code.
    # Agriculture: 01-03
    if (x >= 1) & (x <= 3):
        return "Agriculture"
    # Industry: 05-43
    elif (x >= 5) & (x <= 43):
        return "Industry"
    # Services: 45-99
    elif (x >= 45) & (x <= 99):
        return "Services"

In [73]:
### Apply the function to the dataframe.
LFS_56_Q1["SECTOR"] = LFS_56_Q1.apply(sector_create_from55, axis = 1)

LFS_56_Q1.head(10)

,reg,cwt,area,psu_no,ea_set,samset,mounth,yr,hh_no,member,...,wage_ty,amount,approx,bonus,ot,oth_mon,re_wk,re_ed,wgt,SECTOR
0,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,11.0,1065.8194,Services
1,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,926.1319,Services
2,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,1.0,3.0,...,4.0,NaN,6000.0,0.0,0.0,0.0,1.0,3.0,1327.5656,Services
3,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,713.0996,Services
5,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,11.0,1065.8194,Services
6,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,11.0,1134.2707,Services
8,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,926.1319,Services
9,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,1065.8194,Industry
10,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,926.1319,Industry


### Recoding Education

Value labels of 're_ed': 'EDUCATION RECODE'.

For 2528 - 2543:
- 01-04: Elementary or lower
- 05, 07: Lower secondary education
- 06, 08: Upper secondary or Associate
- 09-10: Bachelor or higher
- if 11
    + if *occup* is >= 71 & <= 74, then Upper secondary or Associate
    + if *occup* is >= 75 & <= 78, then Bachelor or higher
- 12-14: Other or unknown education (excluded from the analysis)

For 2544 - 2549:
- 01-03: Elementary or lower
- 04: Lower secondary education
- 05-10: Upper secondary or Associate
- 11-13: Bachelor or higher
- 14-15: Other or unknown education (excluded from the analysis)

For 2550 - recent:
- 01-03: Elementary or lower
- 04: Lower secondary education
- 05-10: Upper secondary or Associate
- 11-15: Bachelor or higher
- 16-17: Other or unknown education (excluded from the analysis)


In [74]:
### Define fuctions to recode re_ed.
## Regroup re_ed to 6 categories (2528 - 2543)
def EDCODE_NEW_28to43(row):
    if row["re_ed"] < 4:
        return "Elementary or lower"
    elif (row["re_ed"] == 5) | (row["re_ed"] == 7):
        return "Lower secondary"
    elif (row["re_ed"] == 6) | (row["re_ed"] == 8):
        return "Upper secondary or Associate"
    elif (row["re_ed"] == 9) | (row["re_ed"] == 10):
        return "Bachelor or higher"
    elif row["re_ed"] == 11:
        if (row["occup"] >= 74) & (row["occup"] <= 74):
            return "Upper secondary or Associate"
        elif (row["occup"] >= 75) & (row["occup"] <= 78):
            return "Bachelor or higher"
    else:
        return "Other/Unknown"

## Regroup re_ed to 6 categories (2544 - 2549)
def EDCODE_NEW_44to49(row):
    if row["re_ed"] < 3:
        return "Elementary or lower"
    elif row["re_ed"] == 4:
        return "Lower secondary"
    elif (row["re_ed"] >= 5) & (row["re_ed"] <= 10):
        return "Upper secondary or Associate"
    elif (row["re_ed"] >= 11) & (row["re_ed"] <= 13):
        return "Bachelor or higher"
    else: 
        return "Other/Unknown"

## Regroup re_ed to 6 categories (2550 - recent)
def EDCODE_NEW_50(row):
    if row["re_ed"] < 3:
        return "Elementary or lower"
    elif row["re_ed"] == 4:
        return "Lower secondary"
    elif (row["re_ed"] >= 5) & (row["re_ed"] <= 10):
        return "Upper secondary or Associate"
    elif (row["re_ed"] >= 11) & (row["re_ed"] <= 15):
        return "Bachelor or higher"
    else: 
        return "Other/Unknown"

In [75]:
## Apply the function to the dataframe.
LFS_56_Q1["EDCODE_NEW"] = LFS_56_Q1.apply(EDCODE_NEW_50, axis = 1)


In [76]:
LFS_56_Q1

,reg,cwt,area,psu_no,ea_set,samset,mounth,yr,hh_no,member,...,amount,approx,bonus,ot,oth_mon,re_wk,re_ed,wgt,SECTOR,EDCODE_NEW
0,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,1.0,11.0,1065.8194,Services,Bachelor or higher
1,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,1.0,5.0,926.1319,Services,Upper secondary or Associate
2,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,1.0,3.0,...,NaN,6000.0,0.0,0.0,0.0,1.0,3.0,1327.5656,Services,Other/Unknown
3,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other/Unknown
4,1.0,10.0,1.0,151.0,2.0,A,1.0,56.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,713.0996,Services,Elementary or lower
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226608,5.0,96.0,2.0,54.0,3.0,B,3.0,56.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,341.3266,Agriculture,Elementary or lower
226609,5.0,96.0,2.0,54.0,3.0,B,3.0,56.0,6.0,8.0,...,NaN,4500.0,0.0,0.0,0.0,1.0,3.0,383.8208,Services,Other/Unknown
226610,5.0,96.0,2.0,54.0,3.0,B,3.0,56.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,1.0,3.0,476.7949,Agriculture,Other/Unknown
226611,5.0,96.0,2.0,54.0,3.0,B,3.0,56.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,1.0,3.0,503.8714,Agriculture,Other/Unknown


## Get the total number of labor force in each region.

In [77]:
### Table of number of labor force by reg.
LFS_56_Q1_TOTALLF = LFS_56_Q1.groupby("reg")[["wgt"]].sum().reset_index() 

LFS_56_Q1_TOTALLF

,reg,wgt
0,1.0,5.288283e+06
1,2.0,1.150987e+07
2,3.0,6.617623e+06
3,4.0,9.843070e+06
4,5.0,5.243078e+06


In [78]:
## Compute the total labor force at the national level.
LFS_56_Q1_TOTALLF_NAT = LFS_56_Q1_TOTALLF[["wgt"]].sum().reset_index() 

## Create column reg for the national level. Set it to 0. 
LFS_56_Q1_TOTALLF_NAT["reg"] = 0

# drop the index column
LFS_56_Q1_TOTALLF_NAT = LFS_56_Q1_TOTALLF_NAT.drop(columns = "index")

# rename column "0" to wgt
LFS_56_Q1_TOTALLF_NAT = LFS_56_Q1_TOTALLF_NAT.rename(columns = {0: "wgt"})

# Make the order of columns the same as LFS_56_Q1_TOTALLF
LFS_56_Q1_TOTALLF_NAT = LFS_56_Q1_TOTALLF_NAT[["reg", "wgt"]]

LFS_56_Q1_TOTALLF_NAT


,reg,wgt
0,0,3.850192e+07


In [79]:
## Concatenate the national level data to the regional level data.
LFS_56_Q1_TOTALLF = pd.concat([LFS_56_Q1_TOTALLF_NAT, LFS_56_Q1_TOTALLF], axis = 0)

LFS_56_Q1_TOTALLF

,reg,wgt
0,0.0,3.850192e+07
0,1.0,5.288283e+06
1,2.0,1.150987e+07
2,3.0,6.617623e+06
3,4.0,9.843070e+06
4,5.0,5.243078e+06


## Examine employment in the three broad sectors: agriculture, industry, and services.

### Look at workers of all age groups

In [80]:
### Table of the number of people in each sector by region.
## Tabulate the data.
LFS_56_Q1_SEC_REG = LFS_56_Q1.groupby(["reg", "SECTOR"]).agg({"wgt": "sum"}).reset_index() 

## Create a new column for the percentage of each sector.
LFS_56_Q1_SEC_REG["Percentage"] = LFS_56_Q1_SEC_REG.groupby('reg')["wgt"].apply(lambda x: x / x.sum() * 100).reset_index().wgt



In [81]:
## Compute the total number of people in each sector at the national level.
LFS_56_Q1_SEC_NAT = LFS_56_Q1_SEC_REG.groupby("SECTOR").agg({"wgt": "sum"}).reset_index()

## Create column reg for the national level. Set it to 0. 
LFS_56_Q1_SEC_NAT["reg"] = 0

## Create a new column for the percentage of each sector.
LFS_56_Q1_SEC_NAT["Percentage"] = LFS_56_Q1_SEC_NAT["wgt"]/LFS_56_Q1_SEC_NAT["wgt"].sum() * 100

# Make the order of columns the same as LFS_56_Q1_SEC_REG
LFS_56_Q1_SEC_NAT = LFS_56_Q1_SEC_NAT[LFS_56_Q1_SEC_REG.columns]

In [82]:
## Concatenate the national level data to the regional level data.
LFS_56_Q1_SEC_REG = pd.concat([LFS_56_Q1_SEC_NAT, LFS_56_Q1_SEC_REG]).reset_index(drop=True)

## remove the dataframes that are no longer needed.
del LFS_56_Q1_SEC_NAT

LFS_56_Q1_SEC_REG

,reg,SECTOR,wgt,Percentage
0,0.0,Agriculture,1.189235e+07,31.200190
1,0.0,Industry,9.406667e+06,24.678875
2,0.0,Services,1.681726e+07,44.120935
3,1.0,Agriculture,4.002174e+04,0.762425
4,1.0,Industry,1.345019e+06,25.622991
5,1.0,Services,3.864226e+06,73.614583
6,2.0,Agriculture,2.248091e+06,19.598324
7,2.0,Industry,4.202544e+06,36.636787
8,2.0,Services,5.020196e+06,43.764889
9,3.0,Agriculture,2.971219e+06,45.302086


## Examine education level. 

### Look at labor force of all age groups.

In [83]:
### Tabulate the data. Get the number of people in each education level (5 categories), in each region (5 regions).
## Tabulate the data.
LFS_56_Q1_EDU_REG_TOTAL = LFS_56_Q1.groupby(["reg", "EDCODE_NEW"]).agg({"wgt": "sum"}).reset_index() ## Don't sort by the alphabetical order.

## Exclude the "Other/Unknown" category.
LFS_56_Q1_EDU_REG_TOTAL = LFS_56_Q1_EDU_REG_TOTAL[LFS_56_Q1_EDU_REG_TOTAL["EDCODE_NEW"] != "Other/Unknown"].reset_index(drop=True) ## drop previous index (otherwise it becomes a new column)

## Create a new column for the percentage of each education level in each region.
LFS_56_Q1_EDU_REG_TOTAL["Percentage"] = LFS_56_Q1_EDU_REG_TOTAL.groupby("reg")["wgt"].apply(lambda x: x/x.sum()*100).reset_index().wgt



In [84]:
## Compute the total number of people in each education level at the national level
LFS_56_Q1_EDU_NAT_TOTAL = LFS_56_Q1_EDU_REG_TOTAL.groupby(["EDCODE_NEW"]).agg({"wgt": "sum"}).reset_index()

## Create column reg for the national level. Set it to 0. 
LFS_56_Q1_EDU_NAT_TOTAL["reg"] = 0

## Create a new column for the percentage of each education level
LFS_56_Q1_EDU_NAT_TOTAL["Percentage"] = LFS_56_Q1_EDU_NAT_TOTAL["wgt"]/LFS_56_Q1_EDU_NAT_TOTAL["wgt"].sum()*100

# Make the order of columns the same as LFS_56_Q1_EDU_REG_TOTAL
LFS_56_Q1_EDU_NAT_TOTAL = LFS_56_Q1_EDU_NAT_TOTAL[LFS_56_Q1_EDU_REG_TOTAL.columns]


In [85]:
## Concatenate the national level data to the regional level data.
LFS_56_Q1_EDU_REG_TOTAL = pd.concat([LFS_56_Q1_EDU_NAT_TOTAL, LFS_56_Q1_EDU_REG_TOTAL], axis = 0).reset_index(drop=True) ## drop previous index entirely.

In [86]:

## remove the dataframes that are no longer needed.
del LFS_56_Q1_EDU_NAT_TOTAL

In [87]:
### Sort the table by EDCODE_NEW.
custom_dict_edu = {'Elementary or lower': 0, 'Lower secondary': 1, 'Upper secondary or Associate': 2, 'Bachelor or higher': 3, 'Other/Unknown': 4}

# Convert EDCODE_NEW to an ordered categorical data type
LFS_56_Q1_EDU_REG_TOTAL['EDCODE_NEW'] = pd.Categorical(LFS_56_Q1_EDU_REG_TOTAL['EDCODE_NEW'], categories=custom_dict_edu.keys(), ordered=True)

# Sort the table by EDCODE_NEW and reg
LFS_56_Q1_EDU_REG_TOTAL.sort_values(by=['reg', 'EDCODE_NEW'], inplace=True, ignore_index = True)


In [88]:
LFS_56_Q1_EDU_REG_TOTAL

,reg,EDCODE_NEW,wgt,Percentage
0,0.0,Elementary or lower,1.076333e+07,35.921868
1,0.0,Lower secondary,6.033352e+06,20.135897
2,0.0,Upper secondary or Associate,7.512063e+06,25.070995
3,0.0,Bachelor or higher,5.654421e+06,18.871241
4,1.0,Elementary or lower,5.761996e+05,12.886603
5,1.0,Lower secondary,7.914474e+05,17.700582
6,1.0,Upper secondary or Associate,1.236171e+06,27.646749
7,1.0,Bachelor or higher,1.867489e+06,41.766066
8,2.0,Elementary or lower,2.794547e+06,30.696510
9,2.0,Lower secondary,2.066581e+06,22.700218


## Examine Average Wages in Each Sector.

In [89]:
### Compute weigthed average wage for each sector.
LFS_56_Q1_SEC_WAGE = LFS_56_Q1[LFS_56_Q1.approx != 99999].dropna(subset = ["approx"]).groupby(["reg", "SECTOR"]).apply(lambda x: np.average(x["approx"], weights = x["wgt"])).reset_index(name = "WAGE")

## Compute the average wage for each sector at the national level.
LFS_56_Q1_SEC_WAGE_NAT = LFS_56_Q1[LFS_56_Q1.approx != 99999].dropna(subset = ["approx"]).groupby(["SECTOR"]).apply(lambda x: np.average(x["approx"], weights = x["wgt"])).reset_index(name = "WAGE")

## Create column REG_NEW for the national level. Set it to 0.
LFS_56_Q1_SEC_WAGE_NAT["reg"] = 0

## Make the order of columns the same as LFS_56_Q1_SEC_WAGE
LFS_56_Q1_SEC_WAGE_NAT = LFS_56_Q1_SEC_WAGE_NAT[LFS_56_Q1_SEC_WAGE.columns]

## Concatenate the national level data to the regional level data.
LFS_56_Q1_SEC_WAGE = pd.concat([LFS_56_Q1_SEC_WAGE_NAT, LFS_56_Q1_SEC_WAGE]).reset_index(drop=True)

## remove the dataframes that are no longer needed.
del LFS_56_Q1_SEC_WAGE_NAT

LFS_56_Q1_SEC_WAGE

,reg,SECTOR,WAGE
0,0.0,Agriculture,10956.100928
1,0.0,Industry,12108.694673
2,0.0,Services,18443.418570
3,1.0,Agriculture,7294.644236
4,1.0,Industry,18618.056390
5,1.0,Services,22718.034892
6,2.0,Agriculture,11625.916883
7,2.0,Industry,12321.981422
8,2.0,Services,18132.057156
9,3.0,Agriculture,9173.728590


## Setting up functions for streamling the process on data from all periods.

In [90]:
#### Turning the above code into a function. Must apply (pipe) to the dataframe.
### Function for RECODING, EXCLUDING NON-LABOR FORCE (2555 - recent)

def LFS_from55_clean(LFS_df):
    ## Exclude people who are not in labor force.
    LFS_df = LFS_df.pipe(exclude_not_LFfrom44)

    ## Recode SECTOR
    LFS_df["SECTOR"] = LFS_df.apply(sector_create_from55, axis = 1)
        
    ## Recode EDCODE
    LFS_df["EDCODE_NEW"] = LFS_df.apply(EDCODE_NEW_50, axis = 1)
    
    return LFS_df

In [91]:
### Function for RECODING, EXCLUDING NON-LABOR FORCE 2554
def LFS_54_clean(LFS_df):
    ## Exclude people who are not in labor force.
    LFS_df = LFS_df.pipe(exclude_not_LFfrom44)

    ## Recode SECTOR
    LFS_df["SECTOR"] = LFS_df.apply(sector_create_54, axis = 1)

    ## Recode EDCODE
    LFS_df["EDCODE_NEW"] = LFS_df.apply(EDCODE_NEW_50, axis = 1)

    return LFS_df

In [92]:
### Function for RECODING, EXCLUDING NON-LABOR FORCE (2550 - 2553)
def LFS_50to53_clean(LFS_df):
    ## Exclude people who are not in labor force.
    LFS_df = LFS_df.pipe(exclude_not_LFfrom44)

    ## Recode SECTOR
    LFS_df["SECTOR"] = LFS_df.apply(sector_create_44to53, axis = 1)

    ## Recode EDCODE
    LFS_df["EDCODE_NEW"] = LFS_df.apply(EDCODE_NEW_50, axis = 1)

    return LFS_df

In [93]:
### Function for RECODING, EXCLUDING NON-LABOR FORCE (2544 - 2549)
def LFS_44to49_clean(LFS_df):
    ## Exclude people who are not in labor force.
    LFS_df = LFS_df.pipe(exclude_not_LFfrom44)

    ## Recode SECTOR
    LFS_df["SECTOR"] = LFS_df.apply(sector_create_44to53, axis = 1)

    ## Recode EDCODE
    LFS_df["EDCODE_NEW"] = LFS_df.apply(EDCODE_NEW_44to49, axis = 1)

    return LFS_df

In [94]:
### Function for RECODING, EXCLUDING NON-LABOR FORCE (2528 - 2543)
def LFS_28to43_clean(LFS_df):
    ## Exclude people who are not in labor force.
    LFS_df = LFS_df.pipe(exclude_not_LFbefore44)

    ## Recode SECTOR
    LFS_df["SECTOR"] = LFS_df.apply(sector_create_28to43, axis = 1)

    ## Recode EDCODE
    LFS_df["EDCODE_NEW"] = LFS_df.apply(EDCODE_NEW_28to43, axis = 1)

    return LFS_df

In [95]:
### Function for Tabulate sector for workers of all ages.
def TAB_SEC(LFS):
        ### Number of people in each sector by region.
        LFS_SEC_REG = LFS.groupby(["reg", "SECTOR"]).agg({"wgt": "sum"}).reset_index()

        ## Create a new column for the percentage of each sector.
        LFS_SEC_REG["Percentage"] = LFS_SEC_REG.groupby('reg')["wgt"].apply(lambda x: x / x.sum() * 100).reset_index().wgt

        ## Compute the total number of people in each sector at the national level.
        LFS_SEC_NAT = LFS_SEC_REG.groupby("SECTOR").agg({"wgt": "sum"}).reset_index()

        ## Create column reg for the national level. Set it to 0.
        LFS_SEC_NAT["reg"] = 0

        ## Create a new column for the percentage of each sector.
        LFS_SEC_NAT["Percentage"] = LFS_SEC_NAT["wgt"]/LFS_SEC_NAT["wgt"].sum() * 100

        # Make the order of columns the same as LFS_SEC_REG
        LFS_SEC_NAT = LFS_SEC_NAT[LFS_SEC_REG.columns]

        ## Concatenate the national level data to the regional level data.
        LFS_SEC_REG = pd.concat([LFS_SEC_NAT, LFS_SEC_REG]).reset_index(drop=True)

        return LFS_SEC_REG

In [96]:
### Function for Tabulate education level for workers of all ages.
def TAB_EDU(LFS):
        ### Tabulate the data. Get the number of people in each education level, in each region.
        ## Tabulate the data.
        LFS_EDU_REG_TOTAL = LFS.groupby(["reg", "EDCODE_NEW"]).agg({"wgt": "sum"}).reset_index() ## Don't sort by the alphabetical order.

        ## Exclude the "Other/Unknown" category.
        LFS_EDU_REG_TOTAL = LFS_EDU_REG_TOTAL[LFS_EDU_REG_TOTAL["EDCODE_NEW"] != "Other/Unknown"].reset_index(drop=True) ## drop previous index (otherwise it becomes a new column)

        ## Create a new column for the percentage of each education level in each region.
        LFS_EDU_REG_TOTAL["Percentage"] = LFS_EDU_REG_TOTAL.groupby("reg")["wgt"].apply(lambda x: x/x.sum()*100).reset_index().wgt

        ## Compute the total number of people in each education level at the national level
        LFS_EDU_NAT_TOTAL = LFS_EDU_REG_TOTAL.groupby(["EDCODE_NEW"]).agg({"wgt": "sum"}).reset_index()

        ## Create column reg for the national level. Set it to 0.
        LFS_EDU_NAT_TOTAL["reg"] = 0

        ## Create a new column for the percentage of each education level
        LFS_EDU_NAT_TOTAL["Percentage"] = LFS_EDU_NAT_TOTAL["wgt"]/LFS_EDU_NAT_TOTAL["wgt"].sum()*100

        # Make the order of columns the same as LFS_EDU_REG_TOTAL
        LFS_EDU_NAT_TOTAL = LFS_EDU_NAT_TOTAL[LFS_EDU_REG_TOTAL.columns]

        ## Concatenate the national level data to the regional level data.
        LFS_EDU_REG_TOTAL = pd.concat([LFS_EDU_NAT_TOTAL, LFS_EDU_REG_TOTAL], axis = 0).reset_index(drop=True) ## drop previous index entirely.

        ### Sort the table by EDCODE_NEW.
        custom_dict_edu = {'Elementary or lower': 0, 'Lower secondary': 1, 'Upper secondary or Associate': 2, 'Bachelor or higher': 3, 'Other/Unknown': 4}
                           
        # Convert EDCODE_NEW to an ordered categorical data type
        LFS_EDU_REG_TOTAL['EDCODE_NEW'] = pd.Categorical(LFS_EDU_REG_TOTAL['EDCODE_NEW'], categories=custom_dict_edu.keys(), ordered=True)

        # Sort the table by EDCODE_NEW and reg
        LFS_EDU_REG_TOTAL.sort_values(by=['reg', 'EDCODE_NEW'], inplace=True, ignore_index = True)

        return LFS_EDU_REG_TOTAL

In [97]:
### Function for calculating the average wage for each sector.
def avg_wage(LFS):
    ### Compute weigthed average wage for each sector.
    LFS_SEC_WAGE = LFS[LFS.approx != 99999].dropna(subset = ["approx"]).groupby(["reg", "SECTOR"]).apply(lambda x: np.average(x["approx"], weights = x["wgt"])).reset_index(name = "WAGE")

    ## Compute the average wage for each sector at the national level.
    LFS_SEC_WAGE_NAT = LFS[LFS.approx != 99999].dropna(subset = ["approx"]).groupby(["SECTOR"]).apply(lambda x: np.average(x["approx"], weights = x["wgt"])).reset_index(name = "WAGE")

    ## Create column REG_NEW for the national level. Set it to 0.
    LFS_SEC_WAGE_NAT["reg"] = 0

    ## Make the order of columns the same as LFS_SEC_WAGE
    LFS_SEC_WAGE_NAT = LFS_SEC_WAGE_NAT[LFS_SEC_WAGE.columns]

    ## Concatenate the national level data to the regional level data.
    LFS_SEC_WAGE = pd.concat([LFS_SEC_WAGE_NAT, LFS_SEC_WAGE]).reset_index(drop=True)

    return LFS_SEC_WAGE

In [98]:
### Function for calculating the total labor force by region.
def total_LF(LFS):
    ### Table of number of labor force by reg.
    LFS_TOTALLF = LFS.groupby("reg")[["wgt"]].sum().reset_index()

    ## Compute the total labor force at the national level.
    LFS_TOTALLF_NAT = LFS_TOTALLF[["wgt"]].sum().reset_index()

    ## Create column reg for the national level. Set it to 0.
    LFS_TOTALLF_NAT["reg"] = 0

    # drop the index column
    LFS_TOTALLF_NAT = LFS_TOTALLF_NAT.drop(columns = "index")

    # rename column "0" to wgt
    LFS_TOTALLF_NAT = LFS_TOTALLF_NAT.rename(columns = {0: "wgt"})

    # Make the order of columns the same as LFS_TOTALLF
    LFS_TOTALLF_NAT = LFS_TOTALLF_NAT[["reg", "wgt"]]

    ## Concatenate the national level data to the regional level data.
    LFS_TOTALLF = pd.concat([LFS_TOTALLF_NAT, LFS_TOTALLF], axis = 0)

    return LFS_TOTALLF

In [99]:
total_LF(LFS_56_Q1)

,reg,wgt
0,0.0,3.850192e+07
0,1.0,5.288283e+06
1,2.0,1.150987e+07
2,3.0,6.617623e+06
3,4.0,9.843070e+06
4,5.0,5.243078e+06


In [100]:
### From all years up until 2540, REG is coded differently. 1 used to be North, 2 used to be Northeast, 3 used to be South, 4 used to be Central, and 5 used to be Bangkok.
## Therefore, recode REG during 2528 - 2540 to match the coding of REG from 2541 - recent. Replace 1 with 3, 2 with 4, 3 with 5, 4 with 2, and 5 with 1.

def recode_reg_28to40(LFS_df):
    LFS_df["reg"] = LFS_df["reg"].replace({1: 3, 2: 4, 3: 5, 4: 2, 5: 1})
    return LFS_df

## Repeat the same process for data from 2544 to 2563 all quarters. (Have approx in all quarters)

In [101]:
# Creating a dictionary to store tabulation results (dataframes).
LFS_44to63_ALL_AGE_SEC = {}
LFS_44to63_ALL_AGE_EDU = {}
LFS_44to63_WAGE_SEC = {}
LFS_44to63_TOTALLF = {}


### Import Labor force survey
list_year = [2544, 2545, 2546, 2547, 2548, 2549, 2550, 2551, 2552, 2553, 2554, 2555, 2556, 2557, 2558, 2559, 2560, 2561, 2562, 2563, 2564, 2565]
list_quarter = [1, 2, 3, 4]

root_path = "Data\LFS"

for year in list_year:
    for quarter in list_quarter:
        year_CE = year -543
        filename = root_path + "\LFS " + str(year) + " (" + str(year_CE) + ")\LFS Q" + str(quarter) + " " + str(year) + "\Microdata LFS Q" + str(quarter) + " " + str(year) + "\Microdata LFS Q" + str(quarter) + " " + str(year) + ".SAV"
        Data_Time = "LFS_" + str(year) + "_Q" + str(quarter)
        LFS, meta = pyreadstat.read_sav(filename)

        ### Rename to stabilize column names.
        ## REG -> reg
        LFS.rename(columns = {"REG": "reg"}, inplace = True)
        ## wt -> wgt
        LFS.rename(columns = {"wt": "wgt"}, inplace = True)
        ## Weight -> wgt
        LFS.rename(columns = {"Weight": "wgt"}, inplace = True)
        ## WEIGHT -> wgt
        LFS.rename(columns = {"WEIGHT": "wgt"}, inplace = True)
        ## WT -> wgt
        LFS.rename(columns = {"WT": "wgt"}, inplace = True)
        ## WGT -> wgt
        LFS.rename(columns = {"WGT": "wgt"}, inplace = True)
        ## Weight1 -> wgt
        LFS.rename(columns = {"Weight1": "wgt"}, inplace = True)
        ## WGT_REG -> wgt
        LFS.rename(columns = {"WGT_REG": "wgt"}, inplace = True)
        ## RE_WK -> re_wk
        LFS.rename(columns = {"RE_WK": "re_wk"}, inplace = True)
        ## WKCODE -> re_wk
        LFS.rename(columns = {"WKCODE": "re_wk"}, inplace = True)
        ## RE_ED -> re_ed
        LFS.rename(columns = {"RE_ED": "re_ed"}, inplace = True)
        ## EDCODE -> re_ed
        LFS.rename(columns = {"EDCODE": "re_ed"}, inplace = True)
        ## CWT -> CWD
        LFS.rename(columns = {"CWT": "CWD"}, inplace = True)
        ## INDUS -> indus
        LFS.rename(columns = {"INDUS": "indus"}, inplace = True)
        ## INDUST -> indus
        LFS.rename(columns = {"INDUST": "indus"}, inplace = True)
        ## INDUSTRY -> indus
        LFS.rename(columns = {"INDUSTRY": "indus"}, inplace = True)
        ## OCCPTN -> occup
        LFS.rename(columns = {"OCCPTN": "occup"}, inplace = True)
        ## APPROX -> approx
        LFS.rename(columns = {"APPROX": "approx"}, inplace = True)
        ## PERMONTH -> approx
        LFS.rename(columns = {"PERMONTH": "approx"}, inplace = True)
        ## AGE -> age
        LFS.rename(columns = {"AGE": "age"}, inplace = True)

        ###### Apply created functions.
        ### Clean the data.
        if year >= 2555:
            LFS = LFS.pipe(LFS_from55_clean)
        elif year == 2554:
            LFS = LFS.pipe(LFS_54_clean)
        elif (year >= 2550 and year <= 2553):
            LFS = LFS.pipe(LFS_50to53_clean)
        else:
            LFS = LFS.pipe(LFS_44to49_clean)

        
        ### Very special case, When year is 2554 or 2555 and quarter is 3 or 4, weight must be multiplied by 10,000.
        if (year == 2554 or year == 2555) and (quarter == 3 or quarter == 4):
            LFS["wgt"] = LFS["wgt"] * 10000

        ## Remove rows zero weight.
        LFS = LFS.drop(LFS[LFS["wgt"] <= 0].index)
        ## Remove rows with missing values in the weight column.
        LFS = LFS.dropna(subset = ["wgt"])

        ### Tabulate the data.
        LFS_44to63_ALL_AGE_SEC[Data_Time] = LFS.pipe(TAB_SEC)
        LFS_44to63_ALL_AGE_EDU[Data_Time] = LFS.pipe(TAB_EDU)
        LFS_44to63_TOTALLF[Data_Time] = LFS.pipe(total_LF)

        ### Compute average wage for each sector.
        LFS_44to63_WAGE_SEC[Data_Time] = LFS.pipe(avg_wage)

In [102]:
### Create a function to annualize the quarterly data.
def annualize(LFS_dict, column_name):
    ## Create empty dictionary to store data in each year.
    LFS_annual_data = {}
    for year in list_year:
        quarter = 1
        Value_all_Q = pd.DataFrame()
        while quarter <= 4:
            Data_Time = "LFS_" + str(year) + "_Q" + str(quarter)
            quarterly_data = LFS_dict[Data_Time]

            ## Collect WT of each quarter as a column.
            Value_all_Q[Data_Time] = quarterly_data[column_name]
            ## Move to the next quarter.
            quarter += 1 

        ## Average the quarterly data.
        # Ignore missing values.
        Value_all_Q[column_name] = np.nanmean(Value_all_Q, axis = 1)
        ## Denote the year.
        Value_all_Q["Year"] = year

        ## Concat with the original df to have regions and sectors/edu.
        # If contains both reg and SECTOR columns.
        if "reg" in quarterly_data.columns and "SECTOR" in quarterly_data.columns:
            Value_all_Q = pd.concat([quarterly_data.iloc[:,0:2], Value_all_Q[["Year" ,column_name]]], axis = 1)
        # If contains both reg and EDCODE_NEW columns.
        elif "reg" in quarterly_data.columns and "EDCODE_NEW" in quarterly_data.columns:
            Value_all_Q = pd.concat([quarterly_data.iloc[:,0:2], Value_all_Q[["Year" ,column_name]]], axis = 1)
        # If contains only reg.
        elif "reg" in quarterly_data.columns and "SECTOR" not in quarterly_data.columns:
            Value_all_Q = pd.concat([quarterly_data.iloc[:,0:1], Value_all_Q[["Year" ,column_name]]], axis = 1)

        ## Store the annual data in the dictionary.
        LFS_annual_data[str(year)] = Value_all_Q

    return LFS_annual_data

In [103]:
### Annualize the data. Make dictionaries.
## Total Number.
LFS44to65_ALL_AGE_SEC_annual_num = annualize(LFS_44to63_ALL_AGE_SEC, "wgt")
LFS44to65_ALL_AGE_EDU_annual_num = annualize(LFS_44to63_ALL_AGE_EDU, "wgt")
LFS44to65_TOTALLF_annual_num = annualize(LFS_44to63_TOTALLF, "wgt")

## Percentage.
LFS44to65_ALL_AGE_SEC_annual_pct = annualize(LFS_44to63_ALL_AGE_SEC, "Percentage")
LFS44to65_ALL_AGE_EDU_annual_pct = annualize(LFS_44to63_ALL_AGE_EDU, "Percentage")

## Average wage.
LFS44to65_WAGE_SEC_annual = annualize(LFS_44to63_WAGE_SEC, "WAGE")


## Repeat the same process for data from 2541 to 2543 all quarters. Have 4 quarters. But only the first and the third quarter have data on BAHT_PER_DAY.

### EDU and SECTOR

In [104]:
# Creating a dictionary to store tabulation results (dataframes).
LFS_41to43_ALL_AGE_SEC = {}
LFS_41to43_ALL_AGE_EDU = {}
LFS_41to41_TOTAL_LF = {}

### Import Labor force survey
list_year = [2541, 2542, 2543]
list_quarter = [1, 2, 3, 4]

root_path = "Data\LFS"

for year in list_year:
    for quarter in list_quarter:
        year_CE = year -543
        filename = root_path + "\LFS " + str(year) + " (" + str(year_CE) + ")\LFS Q" + str(quarter) + " " + str(year) + "\Microdata LFS Q" + str(quarter) + " " + str(year) + "\Microdata LFS Q" + str(quarter) + " " + str(year) + ".SAV"
        Data_Time = "LFS_" + str(year) + "_Q" + str(quarter)
        LFS, meta = pyreadstat.read_sav(filename)

        ### Rename to stabilize column names.
        ## REG -> reg
        LFS.rename(columns = {"REG": "reg"}, inplace = True)
        ## wt -> wgt
        LFS.rename(columns = {"wt": "wgt"}, inplace = True)
        ## Weight -> wgt
        LFS.rename(columns = {"Weight": "wgt"}, inplace = True)
        ## WEIGHT -> wgt
        LFS.rename(columns = {"WEIGHT": "wgt"}, inplace = True)
        ## WT -> wgt
        LFS.rename(columns = {"WT": "wgt"}, inplace = True)
        ## WGT -> wgt
        LFS.rename(columns = {"WGT": "wgt"}, inplace = True)
        ## Weight1 -> wgt
        LFS.rename(columns = {"Weight1": "wgt"}, inplace = True)
        ## WGT_REG -> wgt
        LFS.rename(columns = {"WGT_REG": "wgt"}, inplace = True)
        ## RE_WK -> re_wk
        LFS.rename(columns = {"RE_WK": "re_wk"}, inplace = True)
        ## WKCODE -> re_wk
        LFS.rename(columns = {"WKCODE": "re_wk"}, inplace = True)
        ## RE_ED -> re_ed
        LFS.rename(columns = {"RE_ED": "re_ed"}, inplace = True)
        ## EDCODE -> re_ed
        LFS.rename(columns = {"EDCODE": "re_ed"}, inplace = True)
        ## CWT -> CWD
        LFS.rename(columns = {"CWT": "CWD"}, inplace = True)
        ## INDUS -> indus
        LFS.rename(columns = {"INDUS": "indus"}, inplace = True)
        ## INDUST -> indus
        LFS.rename(columns = {"INDUST": "indus"}, inplace = True)
        ## INDUSTRY -> indus
        LFS.rename(columns = {"INDUSTRY": "indus"}, inplace = True)
        ## OCCPTN -> occup
        LFS.rename(columns = {"OCCPTN": "occup"}, inplace = True)
        ## APPROX -> approx
        LFS.rename(columns = {"APPROX": "approx"}, inplace = True)
        ## PERMONTH -> approx
        LFS.rename(columns = {"PERMONTH": "approx"}, inplace = True)
        ## AGE -> age
        LFS.rename(columns = {"AGE": "age"}, inplace = True)


        ###### Apply created functions.
        ### Clean the data.
        LFS = LFS.pipe(LFS_28to43_clean)

        ## Remove rows zero weight.
        LFS = LFS.drop(LFS[LFS["wgt"] <= 0].index)
        ## Remove rows with missing values in the weight column.
        LFS = LFS.dropna(subset = ["wgt"])

        ### Tabulate the data.
        LFS_41to43_ALL_AGE_SEC[Data_Time] = LFS.pipe(TAB_SEC)
        LFS_41to43_ALL_AGE_EDU[Data_Time] = LFS.pipe(TAB_EDU)
        LFS_41to41_TOTAL_LF[Data_Time] = LFS.pipe(total_LF)


In [105]:
### Annualize the data. Make dictionaries.
## Total Number.
LFS41to43_ALL_AGE_SEC_annual_num = annualize(LFS_41to43_ALL_AGE_SEC, "wgt")
LFS41to43_ALL_AGE_EDU_annual_num = annualize(LFS_41to43_ALL_AGE_EDU, "wgt")
LFS41to43_TOTALLF_annual_num = annualize(LFS_41to41_TOTAL_LF, "wgt")

## Percentage.
LFS41to43_ALL_AGE_SEC_annual_pct = annualize(LFS_41to43_ALL_AGE_SEC, "Percentage")
LFS41to43_ALL_AGE_EDU_annual_pct = annualize(LFS_41to43_ALL_AGE_EDU, "Percentage")

## Repeat the same process for data from 2536. Have 2 rounds (R1 and R3)

### EDU and SECTOR

In [106]:
# Creating a dictionary to store tabulation results (dataframes).
LFS_36_ALL_AGE_SEC = {}
LFS_36_ALL_AGE_EDU = {}
LFS_36_TOTAL_LF = {}

### Import Labor force survey
list_year = [2536]
list_round = [1, 3]

root_path = "Data\LFS"

for year in list_year:
    for round in list_round:
        year_CE = year -543
        filename = root_path + "\LFS " + str(year) + " (" + str(year_CE) + ")\LFS R" + str(round) + " " + str(year) + "\Microdata LFS R" + str(round) + " " + str(year) + "\Microdata LFS R" + str(round) + " " + str(year) + ".SAV"
        Data_Time = "LFS_" + str(year) + "_R" + str(round)
        LFS, meta = pyreadstat.read_sav(filename)

        ### Rename to stabilize column names.
        ## REG -> reg
        LFS.rename(columns = {"REG": "reg"}, inplace = True)
        ## wt -> wgt
        LFS.rename(columns = {"wt": "wgt"}, inplace = True)
        ## Weight -> wgt
        LFS.rename(columns = {"Weight": "wgt"}, inplace = True)
        ## WEIGHT -> wgt
        LFS.rename(columns = {"WEIGHT": "wgt"}, inplace = True)
        ## WT -> wgt
        LFS.rename(columns = {"WT": "wgt"}, inplace = True)
        ## WGT -> wgt
        LFS.rename(columns = {"WGT": "wgt"}, inplace = True)
        ## Weight1 -> wgt
        LFS.rename(columns = {"Weight1": "wgt"}, inplace = True)
        ## WGT_REG -> wgt
        LFS.rename(columns = {"WGT_REG": "wgt"}, inplace = True)
        ## RE_WK -> re_wk
        LFS.rename(columns = {"RE_WK": "re_wk"}, inplace = True)
        ## WKCODE -> re_wk
        LFS.rename(columns = {"WKCODE": "re_wk"}, inplace = True)
        ## RE_ED -> re_ed
        LFS.rename(columns = {"RE_ED": "re_ed"}, inplace = True)
        ## EDCODE -> re_ed
        LFS.rename(columns = {"EDCODE": "re_ed"}, inplace = True)
        ## CWT -> CWD
        LFS.rename(columns = {"CWT": "CWD"}, inplace = True)
        ## INDUS -> indus
        LFS.rename(columns = {"INDUS": "indus"}, inplace = True)
        ## INDUST -> indus
        LFS.rename(columns = {"INDUST": "indus"}, inplace = True)
        ## INDUSTRY -> indus
        LFS.rename(columns = {"INDUSTRY": "indus"}, inplace = True)
        ## OCCPTN -> occup
        LFS.rename(columns = {"OCCPTN": "occup"}, inplace = True)
        ## APPROX -> approx
        LFS.rename(columns = {"APPROX": "approx"}, inplace = True)
        ## PERMONTH -> approx
        LFS.rename(columns = {"PERMONTH": "approx"}, inplace = True)
        ## AGE -> age
        LFS.rename(columns = {"AGE": "age"}, inplace = True)
        
        ###### Apply created functions.
        ## Recode REG to match the recent coding.
        LFS = LFS.pipe(recode_reg_28to40)

        ### Clean the data.
        LFS = LFS.pipe(LFS_28to43_clean)

        ## Remove rows zero weight.
        LFS = LFS.drop(LFS[LFS["wgt"] <= 0].index)
        ## Remove rows with missing values in the weight column.
        LFS = LFS.dropna(subset = ["wgt"])

        ### Tabulate the data.
        LFS_36_ALL_AGE_SEC[Data_Time] = LFS.pipe(TAB_SEC)
        LFS_36_ALL_AGE_EDU[Data_Time] = LFS.pipe(TAB_EDU)
        LFS_36_TOTAL_LF[Data_Time] = LFS.pipe(total_LF)

In [107]:
### Create a function to annualize the 2 rounds data.
def annualize_2rounds(LFS_dict, column_name):
    ## Create empty dictionary to store data in each year.
    LFS_annual_data = {}
    for year in list_year:
        round = 1
        Value_all_R = pd.DataFrame()
        for round in list_round:
            Data_Time = "LFS_" + str(year) + "_R" + str(round)
            round_data = LFS_dict[Data_Time]

            ## Collect WT of each round as a column.
            Value_all_R[Data_Time] = round_data[column_name]
            
        ## Average the 2 rounds data.
        # Ignore missing values.
        Value_all_R[column_name] = np.nanmean(Value_all_R, axis = 1)

        ## Denote the year.
        Value_all_R["Year"] = year

        ## Concat with the original df to have regions and sectors/edu.
        # If contains both reg and SECTOR columns.
        if "reg" in round_data.columns and "SECTOR" in round_data.columns:
            Value_all_R = pd.concat([round_data.iloc[:,0:2], Value_all_R[["Year" ,column_name]]], axis = 1)
        # If contains both reg and EDCODE_NEW columns.
        elif "reg" in round_data.columns and "EDCODE_NEW" in round_data.columns:
            Value_all_R = pd.concat([round_data.iloc[:,0:2], Value_all_R[["Year" ,column_name]]], axis = 1)
        # If contains only reg.
        elif "reg" in round_data.columns and "SECTOR" not in round_data.columns:
            Value_all_R = pd.concat([round_data.iloc[:,0:1], Value_all_R[["Year" ,column_name]]], axis = 1)

        ## Store the annual data in the dictionary.
        LFS_annual_data[str(year)] = Value_all_R

    return LFS_annual_data

In [108]:
### Annualize the data. Make dictionaries.
## Total Number.
LFS36_ALL_AGE_SEC_annual_num = annualize_2rounds(LFS_36_ALL_AGE_SEC, "wgt")
LFS36_ALL_AGE_EDU_annual_num = annualize_2rounds(LFS_36_ALL_AGE_EDU, "wgt")
LFS36_TOTAL_LF_annual_num = annualize_2rounds(LFS_36_TOTAL_LF, "wgt")

## Percentage.
LFS36_ALL_AGE_SEC_annual_pct = annualize_2rounds(LFS_36_ALL_AGE_SEC, "Percentage")
LFS36_ALL_AGE_EDU_annual_pct = annualize_2rounds(LFS_36_ALL_AGE_EDU, "Percentage")

## Repeat the same process for data from 2537. Have 3 rounds (R1, R2, and R3)

### EDU and SECTOR

In [109]:
# Creating a dictionary to store tabulation results (dataframes).
LFS_37_ALL_AGE_SEC = {}
LFS_37_ALL_AGE_EDU = {}
LFS_37_TOTAL_LF = {}

### Import Labor force survey

list_year = [2537]
list_round = [1, 2, 3]

root_path = "Data\LFS"

for year in list_year:
    for round in list_round:
        year_CE = year -543
        filename = root_path + "\LFS " + str(year) + " (" + str(year_CE) + ")\LFS R" + str(round) + " " + str(year) + "\Microdata LFS R" + str(round) + " " + str(year) + "\Microdata LFS R" + str(round) + " " + str(year) + ".SAV"
        Data_Time = "LFS_" + str(year) + "_R" + str(round)
        LFS, meta = pyreadstat.read_sav(filename)

        ### Rename to stabilize column names.
        ## REG -> reg
        LFS.rename(columns = {"REG": "reg"}, inplace = True)
        ## wt -> wgt
        LFS.rename(columns = {"wt": "wgt"}, inplace = True)
        ## Weight -> wgt
        LFS.rename(columns = {"Weight": "wgt"}, inplace = True)
        ## WEIGHT -> wgt
        LFS.rename(columns = {"WEIGHT": "wgt"}, inplace = True)
        ## WT -> wgt
        LFS.rename(columns = {"WT": "wgt"}, inplace = True)
        ## WGT -> wgt
        LFS.rename(columns = {"WGT": "wgt"}, inplace = True)
        ## Weight1 -> wgt
        LFS.rename(columns = {"Weight1": "wgt"}, inplace = True)
        ## WGT_REG -> wgt
        LFS.rename(columns = {"WGT_REG": "wgt"}, inplace = True)
        ## RE_WK -> re_wk
        LFS.rename(columns = {"RE_WK": "re_wk"}, inplace = True)
        ## WKCODE -> re_wk
        LFS.rename(columns = {"WKCODE": "re_wk"}, inplace = True)
        ## RE_ED -> re_ed
        LFS.rename(columns = {"RE_ED": "re_ed"}, inplace = True)
        ## EDCODE -> re_ed
        LFS.rename(columns = {"EDCODE": "re_ed"}, inplace = True)
        ## CWT -> CWD
        LFS.rename(columns = {"CWT": "CWD"}, inplace = True)
        ## INDUS -> indus
        LFS.rename(columns = {"INDUS": "indus"}, inplace = True)
        ## INDUST -> indus
        LFS.rename(columns = {"INDUST": "indus"}, inplace = True)
        ## INDUSTRY -> indus
        LFS.rename(columns = {"INDUSTRY": "indus"}, inplace = True)
        ## OCCPTN -> occup
        LFS.rename(columns = {"OCCPTN": "occup"}, inplace = True)
        ## APPROX -> approx
        LFS.rename(columns = {"APPROX": "approx"}, inplace = True)
        ## PERMONTH -> approx
        LFS.rename(columns = {"PERMONTH": "approx"}, inplace = True)
        ## AGE -> age
        LFS.rename(columns = {"AGE": "age"}, inplace = True)

        ###### Apply created functions.
        ## Recode REG to match the recent coding.
        LFS = LFS.pipe(recode_reg_28to40)
        
        ### Clean the data.
        LFS = LFS.pipe(LFS_28to43_clean)

        ## Remove rows zero weight.
        LFS = LFS.drop(LFS[LFS["wgt"] <= 0].index)
        ## Remove rows with missing values in the weight column.
        LFS = LFS.dropna(subset = ["wgt"])

        ### Tabulate the data.
        LFS_37_ALL_AGE_SEC[Data_Time] = LFS.pipe(TAB_SEC)
        LFS_37_ALL_AGE_EDU[Data_Time] = LFS.pipe(TAB_EDU)
        LFS_37_TOTAL_LF[Data_Time] = LFS.pipe(total_LF)


In [110]:
### Create a function to annualize the 3 rounds data.
def annualize_3rounds(LFS_dict, column_name):
    ## Create empty dictionary to store data in each year.
    LFS_annual_data = {}
    for year in list_year:
        round = 1
        Value_all_R = pd.DataFrame()
        for round in list_round:
            Data_Time = "LFS_" + str(year) + "_R" + str(round)
            round_data = LFS_dict[Data_Time]

            ## Collect WT of each round as a column.
            Value_all_R[Data_Time] = round_data[column_name]

        ## Average the 3 rounds data.
        Value_all_R[column_name] = np.nanmean(Value_all_R, axis = 1)

        ## Denote the year.
        Value_all_R["Year"] = year

        ## Concat with the original df to have regions and sectors/edu.
        # If contains both reg and SECTOR columns.
        if "reg" in round_data.columns and "SECTOR" in round_data.columns:
            Value_all_R = pd.concat([round_data.iloc[:,0:2], Value_all_R[["Year" ,column_name]]], axis = 1)
        # If contains both reg and EDCODE_NEW columns.
        elif "reg" in round_data.columns and "EDCODE_NEW" in round_data.columns:
            Value_all_R = pd.concat([round_data.iloc[:,0:2], Value_all_R[["Year" ,column_name]]], axis = 1)
        # If contains only reg.
        elif "reg" in round_data.columns and "SECTOR" not in round_data.columns:
            Value_all_R = pd.concat([round_data.iloc[:,0:1], Value_all_R[["Year" ,column_name]]], axis = 1)

        ## Store the annual data in the dictionary.
        LFS_annual_data[str(year)] = Value_all_R

    return LFS_annual_data


In [111]:
### Annualize the data. Make dictionaries.
## Total Number.
LFS37_ALL_AGE_SEC_annual_num = annualize_3rounds(LFS_37_ALL_AGE_SEC, "wgt")
LFS37_ALL_AGE_EDU_annual_num = annualize_3rounds(LFS_37_ALL_AGE_EDU, "wgt")
LFS37_TOTAL_LF_annual_num = annualize_3rounds(LFS_37_TOTAL_LF, "wgt")

## Percentage.
LFS37_ALL_AGE_SEC_annual_pct = annualize_3rounds(LFS_37_ALL_AGE_SEC, "Percentage")
LFS37_ALL_AGE_EDU_annual_pct = annualize_3rounds(LFS_37_ALL_AGE_EDU, "Percentage")

## Repeat the same process for data from 2538. Have 2 rounds (R1 and R3)

### EDU and SECTOR

In [112]:
# Creating a dictionary to store tabulation results (dataframes).
LFS_38_ALL_AGE_SEC = {}
LFS_38_ALL_AGE_EDU = {}
LFS_38_TOTAL_LF = {}

### Import Labor force survey
list_year = [2538]
list_round = [1, 3]

root_path = "Data\LFS"

for year in list_year:
    for round in list_round:
        year_CE = year -543
        filename = root_path + "\LFS " + str(year) + " (" + str(year_CE) + ")\LFS R" + str(round) + " " + str(year) + "\Microdata LFS R" + str(round) + " " + str(year) + "\Microdata LFS R" + str(round) + " " + str(year) + ".SAV"
        Data_Time = "LFS_" + str(year) + "_R" + str(round)

        LFS, meta = pyreadstat.read_sav(filename)

        ### Rename to stabilize column names.
        ## REG -> reg
        LFS.rename(columns = {"REG": "reg"}, inplace = True)
        ## wt -> wgt
        LFS.rename(columns = {"wt": "wgt"}, inplace = True)
        ## Weight -> wgt
        LFS.rename(columns = {"Weight": "wgt"}, inplace = True)
        ## WEIGHT -> wgt
        LFS.rename(columns = {"WEIGHT": "wgt"}, inplace = True)
        ## WT -> wgt
        LFS.rename(columns = {"WT": "wgt"}, inplace = True)
        ## WGT -> wgt
        LFS.rename(columns = {"WGT": "wgt"}, inplace = True)
        ## Weight1 -> wgt
        LFS.rename(columns = {"Weight1": "wgt"}, inplace = True)
        ## WGT_REG -> wgt
        LFS.rename(columns = {"WGT_REG": "wgt"}, inplace = True)
        ## RE_WK -> re_wk
        LFS.rename(columns = {"RE_WK": "re_wk"}, inplace = True)
        ## WKCODE -> re_wk
        LFS.rename(columns = {"WKCODE": "re_wk"}, inplace = True)
        ## RE_ED -> re_ed
        LFS.rename(columns = {"RE_ED": "re_ed"}, inplace = True)
        ## EDCODE -> re_ed
        LFS.rename(columns = {"EDCODE": "re_ed"}, inplace = True)
        ## CWT -> CWD
        LFS.rename(columns = {"CWT": "CWD"}, inplace = True)
        ## INDUS -> indus
        LFS.rename(columns = {"INDUS": "indus"}, inplace = True)
        ## INDUST -> indus
        LFS.rename(columns = {"INDUST": "indus"}, inplace = True)
        ## INDUSTRY -> indus
        LFS.rename(columns = {"INDUSTRY": "indus"}, inplace = True)
        ## OCCPTN -> occup
        LFS.rename(columns = {"OCCPTN": "occup"}, inplace = True)
        ## APPROX -> approx
        LFS.rename(columns = {"APPROX": "approx"}, inplace = True)
        ## PERMONTH -> approx
        LFS.rename(columns = {"PERMONTH": "approx"}, inplace = True)
        ## AGE -> age
        LFS.rename(columns = {"AGE": "age"}, inplace = True)

        ###### Apply created functions.
        ## Recode REG to match the recent coding.
        LFS = LFS.pipe(recode_reg_28to40)

        ### Clean the data.
        LFS = LFS.pipe(LFS_28to43_clean)

        ## Remove rows zero weight.
        LFS = LFS.drop(LFS[LFS["wgt"] <= 0].index)
        ## Remove rows with missing values in the weight column.
        LFS = LFS.dropna(subset = ["wgt"])

        ### Tabulate the data.
        LFS_38_ALL_AGE_SEC[Data_Time] = LFS.pipe(TAB_SEC)
        LFS_38_ALL_AGE_EDU[Data_Time] = LFS.pipe(TAB_EDU)
        LFS_38_TOTAL_LF[Data_Time] = LFS.pipe(total_LF)
        

In [113]:
### Annualize the data. Make dictionaries.
## Total Number.
LFS38_ALL_AGE_SEC_annual_num = annualize_2rounds(LFS_38_ALL_AGE_SEC, "wgt")
LFS38_ALL_AGE_EDU_annual_num = annualize_2rounds(LFS_38_ALL_AGE_EDU, "wgt")
LFS38_TOTAL_LF_annual_num = annualize_2rounds(LFS_38_TOTAL_LF, "wgt")

## Percentage.
LFS38_ALL_AGE_SEC_annual_pct = annualize_2rounds(LFS_38_ALL_AGE_SEC, "Percentage")
LFS38_ALL_AGE_EDU_annual_pct = annualize_2rounds(LFS_38_ALL_AGE_EDU, "Percentage")

## Repeat the same process for data from 2539. Have 3 rounds (R1, R2, and R3)

### EDU and SECTOR

In [114]:
# Creating a dictionary to store tabulation results (dataframes).
LFS_39_ALL_AGE_SEC = {}
LFS_39_ALL_AGE_EDU = {}
LFS_39_TOTAL_LF = {}

### Import Labor force survey
list_year = [2539]
list_round = [1, 2, 3]

root_path = "Data\LFS"
for year in list_year:
    for round in list_round:
        year_CE = year -543
        filename = root_path + "\LFS " + str(year) + " (" + str(year_CE) + ")\LFS R" + str(round) + " " + str(year) + "\Microdata LFS R" + str(round) + " " + str(year) + "\Microdata LFS R" + str(round) + " " + str(year) + ".SAV"
        Data_Time = "LFS_" + str(year) + "_R" + str(round)

        LFS, meta = pyreadstat.read_sav(filename)

        ### Rename to stabilize column names.
        ## REG -> reg
        LFS.rename(columns = {"REG": "reg"}, inplace = True)
        ## wt -> wgt
        LFS.rename(columns = {"wt": "wgt"}, inplace = True)
        ## Weight -> wgt
        LFS.rename(columns = {"Weight": "wgt"}, inplace = True)
        ## WEIGHT -> wgt
        LFS.rename(columns = {"WEIGHT": "wgt"}, inplace = True)
        ## WT -> wgt
        LFS.rename(columns = {"WT": "wgt"}, inplace = True)
        ## WGT -> wgt
        LFS.rename(columns = {"WGT": "wgt"}, inplace = True)
        ## Weight1 -> wgt
        LFS.rename(columns = {"Weight1": "wgt"}, inplace = True)
        ## WGT_REG -> wgt
        LFS.rename(columns = {"WGT_REG": "wgt"}, inplace = True)
        ## RE_WK -> re_wk
        LFS.rename(columns = {"RE_WK": "re_wk"}, inplace = True)
        ## WKCODE -> re_wk
        LFS.rename(columns = {"WKCODE": "re_wk"}, inplace = True)
        ## RE_ED -> re_ed
        LFS.rename(columns = {"RE_ED": "re_ed"}, inplace = True)
        ## EDCODE -> re_ed
        LFS.rename(columns = {"EDCODE": "re_ed"}, inplace = True)
        ## CWT -> CWD
        LFS.rename(columns = {"CWT": "CWD"}, inplace = True)
        ## INDUS -> indus
        LFS.rename(columns = {"INDUS": "indus"}, inplace = True)
        ## INDUST -> indus
        LFS.rename(columns = {"INDUST": "indus"}, inplace = True)
        ## INDUSTRY -> indus
        LFS.rename(columns = {"INDUSTRY": "indus"}, inplace = True)
        ## OCCPTN -> occup
        LFS.rename(columns = {"OCCPTN": "occup"}, inplace = True)
        ## APPROX -> approx
        LFS.rename(columns = {"APPROX": "approx"}, inplace = True)
        ## PERMONTH -> approx
        LFS.rename(columns = {"PERMONTH": "approx"}, inplace = True)
        ## AGE -> age
        LFS.rename(columns = {"AGE": "age"}, inplace = True)

        ###### Apply created functions.
        ## Recode REG to match the recent coding.
        LFS = LFS.pipe(recode_reg_28to40)

        ### Clean the data.
        LFS = LFS.pipe(LFS_28to43_clean)

        ## Remove rows zero weight.
        LFS = LFS.drop(LFS[LFS["wgt"] <= 0].index)
        ## Remove rows with missing values in the weight column.
        LFS = LFS.dropna(subset = ["wgt"])

        ### Tabulate the data.
        LFS_39_ALL_AGE_SEC[Data_Time] = LFS.pipe(TAB_SEC)
        LFS_39_ALL_AGE_EDU[Data_Time] = LFS.pipe(TAB_EDU)
        LFS_39_TOTAL_LF[Data_Time] = LFS.pipe(total_LF)

In [115]:
### Annualize the data. Make dictionaries.
## Total Number.
LFS39_ALL_AGE_SEC_annual_num = annualize_3rounds(LFS_39_ALL_AGE_SEC, "wgt")
LFS39_ALL_AGE_EDU_annual_num = annualize_3rounds(LFS_39_ALL_AGE_EDU, "wgt")
LFS39_TOTAL_LF_annual_num = annualize_3rounds(LFS_39_TOTAL_LF, "wgt")

## Percentage.
LFS39_ALL_AGE_SEC_annual_pct = annualize_3rounds(LFS_39_ALL_AGE_SEC, "Percentage")
LFS39_ALL_AGE_EDU_annual_pct = annualize_3rounds(LFS_39_ALL_AGE_EDU, "Percentage")

## Repeat the same process for data from 2540. Have 2 rounds (R1 and R3)

### EDU and SECTOR

In [116]:
# Creating a dictionary to store tabulation results (dataframes).
LFS_40_ALL_AGE_SEC = {}
LFS_40_ALL_AGE_EDU = {}
LFS_40_TOTAL_LF = {}

### Import Labor force survey
list_year = [2540]
list_round = [1, 3]

root_path = "Data\LFS"

for year in list_year:
    for round in list_round:
        year_CE = year -543
        filename = root_path + "\LFS " + str(year) + " (" + str(year_CE) + ")\LFS R" + str(round) + " " + str(year) + "\Microdata LFS R" + str(round) + " " + str(year) + "\Microdata LFS R" + str(round) + " " + str(year) + ".SAV"
        Data_Time = "LFS_" + str(year) + "_R" + str(round)

        LFS, meta = pyreadstat.read_sav(filename)

        ### Rename to stabilize column names.
        ## REG -> reg
        LFS.rename(columns = {"REG": "reg"}, inplace = True)
        ## wt -> wgt
        LFS.rename(columns = {"wt": "wgt"}, inplace = True)
        ## Weight -> wgt
        LFS.rename(columns = {"Weight": "wgt"}, inplace = True)
        ## WEIGHT -> wgt
        LFS.rename(columns = {"WEIGHT": "wgt"}, inplace = True)
        ## WT -> wgt
        LFS.rename(columns = {"WT": "wgt"}, inplace = True)
        ## WGT -> wgt
        LFS.rename(columns = {"WGT": "wgt"}, inplace = True)
        ## Weight1 -> wgt
        LFS.rename(columns = {"Weight1": "wgt"}, inplace = True)
        ## WGT_REG -> wgt
        LFS.rename(columns = {"WGT_REG": "wgt"}, inplace = True)
        ## RE_WK -> re_wk
        LFS.rename(columns = {"RE_WK": "re_wk"}, inplace = True)
        ## WKCODE -> re_wk
        LFS.rename(columns = {"WKCODE": "re_wk"}, inplace = True)
        ## RE_ED -> re_ed
        LFS.rename(columns = {"RE_ED": "re_ed"}, inplace = True)
        ## EDCODE -> re_ed
        LFS.rename(columns = {"EDCODE": "re_ed"}, inplace = True)
        ## CWT -> CWD
        LFS.rename(columns = {"CWT": "CWD"}, inplace = True)
        ## INDUS -> indus
        LFS.rename(columns = {"INDUS": "indus"}, inplace = True)
        ## INDUST -> indus
        LFS.rename(columns = {"INDUST": "indus"}, inplace = True)
        ## INDUSTRY -> indus
        LFS.rename(columns = {"INDUSTRY": "indus"}, inplace = True)
        ## OCCPTN -> occup
        LFS.rename(columns = {"OCCPTN": "occup"}, inplace = True)
        ## APPROX -> approx
        LFS.rename(columns = {"APPROX": "approx"}, inplace = True)
        ## PERMONTH -> approx
        LFS.rename(columns = {"PERMONTH": "approx"}, inplace = True)
        ## AGE -> age
        LFS.rename(columns = {"AGE": "age"}, inplace = True)

        ###### Apply created functions.
        ## Recode REG to match the recent coding.
        LFS = LFS.pipe(recode_reg_28to40)

        ### Clean the data.
        LFS = LFS.pipe(LFS_28to43_clean)

        ## Remove rows zero weight.
        LFS = LFS.drop(LFS[LFS["wgt"] <= 0].index)
        ## Remove rows with missing values in the weight column.
        LFS = LFS.dropna(subset = ["wgt"])

        ### Tabulate the data.
        LFS_40_ALL_AGE_SEC[Data_Time] = LFS.pipe(TAB_SEC)
        LFS_40_ALL_AGE_EDU[Data_Time] = LFS.pipe(TAB_EDU)
        LFS_40_TOTAL_LF[Data_Time] = LFS.pipe(total_LF)
        

In [117]:
### Annualize the data. Make dictionaries.
## Total Number.
LFS40_ALL_AGE_SEC_annual_num = annualize_2rounds(LFS_40_ALL_AGE_SEC, "wgt")
LFS40_ALL_AGE_EDU_annual_num = annualize_2rounds(LFS_40_ALL_AGE_EDU, "wgt")
LFS40_TOTAL_LF_annual_num = annualize_2rounds(LFS_40_TOTAL_LF, "wgt")

## Percentage.
LFS40_ALL_AGE_SEC_annual_pct = annualize_2rounds(LFS_40_ALL_AGE_SEC, "Percentage")
LFS40_ALL_AGE_EDU_annual_pct = annualize_2rounds(LFS_40_ALL_AGE_EDU, "Percentage")

## Combine dictionaries of all years into one.

In [118]:
### Combine dictionaries into one.
LFS_ALL_AGE_SEC_annual_num = {**LFS36_ALL_AGE_SEC_annual_num, **LFS37_ALL_AGE_SEC_annual_num, **LFS38_ALL_AGE_SEC_annual_num, **LFS39_ALL_AGE_SEC_annual_num, **LFS40_ALL_AGE_SEC_annual_num, **LFS41to43_ALL_AGE_SEC_annual_num, **LFS44to65_ALL_AGE_SEC_annual_num}
LFS_ALL_AGE_EDU_annual_num = {**LFS36_ALL_AGE_EDU_annual_num, **LFS37_ALL_AGE_EDU_annual_num, **LFS38_ALL_AGE_EDU_annual_num, **LFS39_ALL_AGE_EDU_annual_num, **LFS40_ALL_AGE_EDU_annual_num, **LFS41to43_ALL_AGE_EDU_annual_num, **LFS44to65_ALL_AGE_EDU_annual_num}
LFS_ALL_AGE_SEC_annual_pct = {**LFS36_ALL_AGE_SEC_annual_pct, **LFS37_ALL_AGE_SEC_annual_pct, **LFS38_ALL_AGE_SEC_annual_pct, **LFS39_ALL_AGE_SEC_annual_pct, **LFS40_ALL_AGE_SEC_annual_pct, **LFS41to43_ALL_AGE_SEC_annual_pct, **LFS44to65_ALL_AGE_SEC_annual_pct}
LFS_ALL_AGE_EDU_annual_pct = {**LFS36_ALL_AGE_EDU_annual_pct, **LFS37_ALL_AGE_EDU_annual_pct, **LFS38_ALL_AGE_EDU_annual_pct, **LFS39_ALL_AGE_EDU_annual_pct, **LFS40_ALL_AGE_EDU_annual_pct, **LFS41to43_ALL_AGE_EDU_annual_pct, **LFS44to65_ALL_AGE_EDU_annual_pct}
LFS_ALL_YEAR_TOTAL_LF = {**LFS36_TOTAL_LF_annual_num, **LFS37_TOTAL_LF_annual_num, **LFS38_TOTAL_LF_annual_num, **LFS39_TOTAL_LF_annual_num, **LFS40_TOTAL_LF_annual_num, **LFS41to43_TOTALLF_annual_num, **LFS44to65_TOTALLF_annual_num}

All_YEAR_WAGE_SEC_annual = {**LFS44to65_WAGE_SEC_annual}


In [119]:
LFS44to65_WAGE_SEC_annual["2544"]

,reg,SECTOR,Year,WAGE
0,0.0,Agriculture,2544,2381.890960
1,0.0,Industry,2544,5928.823729
2,0.0,Services,2544,8860.832457
3,1.0,Agriculture,2544,5143.968585
4,1.0,Industry,2544,9065.518135
5,1.0,Services,2544,11820.063606
6,2.0,Agriculture,2544,3109.399476
7,2.0,Industry,2544,6219.768532
8,2.0,Services,2544,8637.286967
9,3.0,Agriculture,2544,2169.150159


In [120]:
LFS44to65_WAGE_SEC_annual["2555"]

,reg,SECTOR,Year,WAGE
0,0.0,Agriculture,2555,9852.736263
1,0.0,Industry,2555,12410.399941
2,0.0,Services,2555,18906.256743
3,1.0,Agriculture,2555,12746.973114
4,1.0,Industry,2555,18952.291134
5,1.0,Services,2555,27839.590741
6,2.0,Agriculture,2555,10083.491841
7,2.0,Industry,2555,12260.946443
8,2.0,Services,2555,19815.301432
9,3.0,Agriculture,2555,9910.348842


In [121]:
### Concat each year's data into one dataframe.
All_YEAR_ALL_AGE_SEC_annual_num_final = pd.concat(LFS_ALL_AGE_SEC_annual_num.values(), ignore_index = True)
All_YEAR_ALL_AGE_EDU_annual_num_final = pd.concat(LFS_ALL_AGE_EDU_annual_num.values(), ignore_index = True)
All_YEAR_ALL_AGE_SEC_annual_pct_final = pd.concat(LFS_ALL_AGE_SEC_annual_pct.values(), ignore_index = True)
All_YEAR_ALL_AGE_EDU_annual_pct_final = pd.concat(LFS_ALL_AGE_EDU_annual_pct.values(), ignore_index = True)
All_YEAR_TOTAL_LF_annual_final = pd.concat(LFS_ALL_YEAR_TOTAL_LF.values(), ignore_index = True)
All_YEAR_WAGE_SEC_annual_final = pd.concat(All_YEAR_WAGE_SEC_annual.values(), ignore_index = True)

## Export to excel

In [122]:
### Export to excel.
## Education level.
All_YEAR_ALL_AGE_EDU_annual_num_final.to_excel("Data\Output\All_YEAR_ALL_AGE_EDU_annual_num_final.xlsx", index = False)
All_YEAR_ALL_AGE_EDU_annual_pct_final.to_excel("Data\Output\All_YEAR_ALL_AGE_EDU_annual_pct_final.xlsx", index = False)

## Sector.
All_YEAR_ALL_AGE_SEC_annual_num_final.to_excel("Data\Output\All_YEAR_ALL_AGE_SEC_annual_num_final.xlsx", index = False)
All_YEAR_ALL_AGE_SEC_annual_pct_final.to_excel("Data\Output\All_YEAR_ALL_AGE_SEC_annual_pct_final.xlsx", index = False)

## Total labor force.
All_YEAR_TOTAL_LF_annual_final.to_excel("Data\Output\All_YEAR_TOTAL_LF_annual_final.xlsx", index = False)

## Wage.
All_YEAR_WAGE_SEC_annual_final.to_excel("Data\Output\All_YEAR_WAGE_SEC_annual_final.xlsx", index = False)